# Feed Forward Neural Network

In [1]:
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import CategoricalCrossentropy

In [1]:
# Preprocessing Parameters
TRAIN_DIR = "../data/train_small/"
MODEL_DIR = "../models/"
VALIDATION_SPLIT = 0.2
IMAGE_SIZE = (128, 128)
SEED = 1

# Training Parameters
NUM_CLASSES = 42
NUM_EPOCHS = 30
BATCH_SIZE = 64
OPTIMIZER = "adam"
LOSS_FN = "categorical_crossentropy"
METRICS = ["accuracy"]

In [3]:
# Defining Model Architecture
def get_model():
    return Sequential([
        Flatten(input_shape = IMAGE_SIZE),
        Dense(128, activation = "relu"),
        Dense(NUM_CLASSES, activation = "softmax")
    ])

## Preprocessing - Grayscale + Augment

Augmentation:
- 45 deg rotation rage
- random horizontal and vertical flip
- 40% zoom range

In [4]:
# Loading Data
image_generator = ImageDataGenerator(
    validation_split = VALIDATION_SPLIT,
    rescale = 1/255,
    rotation_range = 45,
    horizontal_flip = True,
    vertical_flip = True,
    zoom_range = 0.4
)

In [5]:
train_data_gen = image_generator.flow_from_directory(
    color_mode = "grayscale",
    directory = TRAIN_DIR,
    batch_size = BATCH_SIZE,
    target_size = IMAGE_SIZE,
    shuffle = True,
    seed = SEED,
    class_mode = "categorical",
    subset = "training"
)

Found 84331 images belonging to 42 classes.


In [6]:
validation_data_gen = image_generator.flow_from_directory(
    color_mode = "grayscale",
    directory = TRAIN_DIR,
    batch_size = BATCH_SIZE,
    target_size = IMAGE_SIZE,
    shuffle = True,
    seed = SEED,
    class_mode = "categorical",
    subset = "validation"
)

Found 21061 images belonging to 42 classes.


In [7]:
model = get_model()

In [8]:
model.compile(
    optimizer = OPTIMIZER,
    loss = LOSS_FN,
    metrics = METRICS
)

In [ ]:
history = model.fit(
    train_data_gen,
    steps_per_epoch = train_data_gen.n // BATCH_SIZE,
    epochs = NUM_EPOCHS,
    validation_data = validation_data_gen,
    validation_steps = validation_data_gen.n // BATCH_SIZE
)

In [ ]:
acc = history.history["accuracy"]
loss = history.history["loss"]
validation_acc = history.history["validation_accuracy"]
validation_loss = history.history["validation_loss"]

epochs_range = range(EPOCHS)

plt.figure(figsize = (8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label = "Training Accuracy")
plt.plot(epochs_range, validation_acc, label = "Validation Accuracy")
plt.legend(loc = "lower right")
plt.title("Training and Validation Accuracy")

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label = "Training Loss")
plt.plot(epochs_range, validation_loss, label = "Validation Loss")
plt.legend(loc = "lower right")
plt.title("Training and Validation Loss")
plt.show()

In [ ]:
model.save(MODEL_DIR + "ffnn_grayscale_augment.h5")